## Import Statements

In [1]:
import gensim.downloader as api
import numpy as np
import re
import csv
import pandas as pd
import pprint
import string
import nltk
import sys
import spacy

from IPython.display import HTML
from nltk.corpus import wordnet 
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize

pathToDatasets = '../datasets/'
pathToDataScripts = '../datasets/scripts/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'

sys.path.insert(0, pathToDataScripts)
from cleanDataset import tokenize_words 


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

Welcome to cleaning dataset script! 
Would you like to enter a string or clean dataset?
Press 1. for string, 2. for dataset or 3 for nothing!
3
Bye


## Downloading binaries and models


In [2]:
print("Should I reload the model?")
tstString = input()
if("no" in tstString.lower()):
    print(" didnt reload model! ")
else:
    word_vectors = api.load("glove-wiki-gigaword-100")
    nltk.download('vader_lexicon')
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')

Should I reload the model?
yes


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Global Variables and Global Objects

In [3]:
senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;

nlp = spacy.load('en')


NUMBER_OF_ALTERNATIVES = 5
TWEET_START = 100
NUM_OF_TWEETS = 50


VERBOSE_PRINTING = True
# VERBOSE_PRINTING = False

USE_SPACY = False
# USE_SPACY = True


punctuation = r"\"#$%&'+-/;<=>?@[\]*^_`{|}~"

## Class for Sentences


In [4]:
class SentenceWithHTML():
    
    def __init__(self,sentence,html):
        self.sentence = sentence;
        self.html = html;
    
    def getHTML(self):
        return self.html;
    
    def getSentence(self):
        return self.sentence
    
    def setHTML(self, html):
        self.html = html;
    
    def setSentence(self, sentence):
        self.sentence = sentence


class Sentence:
    
    
    def __init__(self, sentence, sentiment):
        self.ogSentence = sentence;
        self.ogSentiment = sentiment;
        self.indexToSetOfWords = {}
        self.alternateSentences = [];
        self.finalShiftSentences = [];

    def addAlternativesByIndex(self, index, listOfAlternatives):
        '''
            Adds the list of possible alternative words that 
            can be used per word based on the index of the word in the tokenized 
            sentence. (from cleanAndTokenizeText())
        '''
        if(self.indexToSetOfWords.get(index)):
            self.indexToSetOfWords[index] = self.indexToSetOfWords.union(set(listOfAlternatives))
        else:
            self.indexToSetOfWords[index] = set(listOfAlternatives)
        
    def addAlternativeStrings(self, strings):
        if(isinstance(strings,str)):
            self.alternateStrings = list(set(self.alternateStrings.append(strings)))
            self.alternateSentences.append(strings)
        else:
            self.alternateSentences.extend(strings)
    

    
    def addFinalSentences(self, sentences):
        if(isinstance(sentences, str)):
            self.finalShiftSentences.append(sentences)
        else:
            self.finalShiftSentences.extend(sentences)

    def resetFinalSentences(self):
        self.finalShiftSentences = [];
        
            

## Utility Code

In [5]:
def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

def cleanAndTokenizeText(text):
    text = text.lower();
    newString = ""
    for char in text:
        if char not in punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

def getPOSTags(tweet):
    if(USE_SPACY == False):
        tags = nltk.pos_tag(tweet)
        return tags;    
    tweet = ' '.join(tweet)
    doc = nlp(tweet)
    tags = [(token.text, token.pos_) for token in doc] # since the format expected is [text,tag]
    return tags;
    

def getAntonymsOfWords(word):
    if(word not in vocabulary):
        return []
    setOfAntonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            anton = l.antonyms()
            if(anton!=[]):
                setOfAntonyms.add(anton[0].name())
    if(len(setOfAntonyms) == 0):
        if(VERBOSE_PRINTING): print("No antonyms found for word {0}".format(word))
    return list(setOfAntonyms)

def listReplacements(word):
    if(word not in vocabulary):
        print(" --- {0} not in vocabulary ---".format(word))
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_ALTERNATIVES)]
    if(possibleReplacements == []):
        print(" --- No replacements for word {0} ---".format(word))
    antonyms = getAntonymsOfWords(word)
    if(antonyms != []):
        possibleReplacements.extend(antonyms)
        if(VERBOSE_PRINTING): print("Some antonyms for word {0} are {1}".format(word, antonyms[:3]))
        return possibleReplacements
    return possibleReplacements
    
def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    if(alternativeWords == []):
        return []
    tempTokens = userTokens[:]
    POSTokens = getPOSTags(tempTokens)
    validWords = []
    
    mainTag = POSTokens[indexOfToken][1]
    mainWord = userTokens[indexOfToken]
    
    for ind,word in enumerate(alternativeWords):
        tempTokens[indexOfToken] = word
        posTags = getPOSTags(tempTokens)
        newTag = (posTags[indexOfToken])[1]
        
        if(str(newTag) == str(mainTag)):
            if(VERBOSE_PRINTING): print("Word {0}[{1}] replaced with {2}[{3}]".format(mainWord, mainTag, word,newTag))
            validWords.append(word)
    if(validWords == [] and VERBOSE_PRINTING):
        print("No POS words found for word {} with tag {}".format(mainWord, mainTag));
    return validWords
        
    
    
def getAlternativeSentences(sentenceObj):
    mainSentence = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;
    
    sentenceTokens = cleanAndTokenizeText(mainSentence)
   
    for ind, word in enumerate(sentenceTokens):
        alternativeSentenceWithHTML = []
        
        score = senty.polarity_scores(word)['compound']
        copyOfTokens = sentenceTokens[:]
        replacements = []
        if(score != 0.0):
            tempReplacements = listReplacements(word) # get embedding based relations
            if(tempReplacements == []):
                print("No replacements found at all for word {0}".format(word))
                continue
            replacements = posApprovedReplacements(tempReplacements[:], copyOfTokens[:], ind)
            if(replacements == []):
                print(" -- No POS approved words! -- for word {0}\n some non-POS:{1}".format(word, tempReplacements[:4]))
                continue
            sentenceObj.addAlternativesByIndex(ind, replacements)
            
            ## Generate new sentences by switching that word
            for newWord in replacements:
                htmlFriendlyTokens = copyOfTokens[:]
                copyOfTokens[ind] = newWord
                htmlFriendlyTokens[ind] = cstr(" <i>[{0}]</i>".format(newWord), "blue");
                newString = ' '.join(copyOfTokens)
                tSentence = SentenceWithHTML(newString, ' '.join(htmlFriendlyTokens))
                alternativeSentenceWithHTML.append(tSentence)
        sentenceObj.addAlternativeStrings(alternativeSentenceWithHTML)
    return sentenceObj

def shiftSentiment(sentenceObj, positive=True):
    
    actualTweet = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;
    alternateTweets = sentenceObj.alternateSentences;
    
    
    happiestTweet = ""
    saddestTweet = ""
    happiestScore = -sys.maxsize - 1
    saddestScore = sys.maxsize
    correctTweets = []
    
    for tSentence in alternateTweets:
        tweet = tSentence.getSentence();
        newSenty = senty.polarity_scores(tweet)['compound']
        
        if(newSenty < saddestScore):
            saddestTweet = tSentence
            saddestScore = newSenty
        if(newSenty > happiestScore):
            happiestTweet = tSentence
            happiestScore = newSenty
            
        if(newSenty == mainSentiment):
            continue
        if(positive == True):
            if(newSenty > mainSentiment):
                correctTweets.append(tSentence)
                continue
            
            elif(newSenty < mainSentiment):
                continue
                # Grab happiest tweet and if it's not "", then generate more happy tweets from it
        if(positive == False):
            if(newSenty < mainSentiment):
                correctTweets.append(tSentence)
                continue
            
            elif(newSenty > mainSentiment):
                continue
                # grab happiest tweet, and if it's not "", then generate more happy tweets from it
                
    if(correctTweets == []):
        print("\n\nNo tweets found when trying to do Positive={}\n\n".format(positive))
    sentenceObj.resetFinalSentences();
    sentenceObj.addFinalSentences(correctTweets);
    return sentenceObj;

In [6]:
def printStrings(sentenceObj):
    newStrings = sentenceObj.finalShiftSentences;
    mainSentiment = sentenceObj.ogSentiment;    
    for ind, tSentence in enumerate(newStrings):
            alteredTweet = tSentence.getSentence()
            htmlText = tSentence.getHTML()
            sentimentOfNewString = senty.polarity_scores(alteredTweet)['compound']
            if(sentimentOfNewString == mainSentiment or sentimentOfNewString == 0):
                display(HTML(cstr("{0}: {1}".format(htmlText,sentimentOfNewString),'DarkGray')))
            elif(sentimentOfNewString > mainSentiment):
                display(HTML(cstr("{0}: {1}".format(htmlText,sentimentOfNewString),'green')))
            else:
                display(HTML(cstr("{0}: {1}".format(htmlText,sentimentOfNewString),'red')))

In [7]:
def runThroughTweets():
    
    tweets_df = pd.read_csv( pathToDatasets + 'cleanedTweets.csv' , nrows=NUM_OF_TWEETS, skiprows=TWEET_START)

    tweets = tweets_df.values

    listOfObjects = []
    for counter,tweet in enumerate(tweets):
        tweet = tweet[0]
        mainSentiment = senty.polarity_scores(tweet)['compound']
        if(mainSentiment == 0):
            continue
        print("\n {0}:{1}\n".format(tweet,mainSentiment))   
        sentenceObj = Sentence(tweet, mainSentiment)
        sentenceObj = getAlternativeSentences(sentenceObj)
        alternateTweets = (sentenceObj.alternateSentences)[:]
        if(alternateTweets == [] or alternateTweets == None):
            print(" -- No new Strings generated ---\n\n")
            continue
        sentenceObj = shiftSentiment(sentenceObj, True);
        printStrings(sentenceObj)
        sentenceObj = shiftSentiment(sentenceObj, False);
        printStrings(sentenceObj)
    
runThroughTweets()


  body of missing northern calif. girl found police have found the remains of a missing northern california girl .. :-0.5267

Some antonyms for word missing are ['have', 'attend', 'hit']
No POS words found for word missing with tag VBG
 -- No POS approved words! -- for word missing
 some non-POS:['dead', 'injured', 'trapped', 'rescued']
Some antonyms for word missing are ['have', 'attend', 'hit']
No POS words found for word missing with tag VBG
 -- No POS approved words! -- for word missing
 some non-POS:['dead', 'injured', 'trapped', 'rescued']
 -- No new Strings generated ---



  i hope they will increase the capacity fast yesterday was such a pain. got the fail whale 15 times in 2 hours.... :-0.3818

Some antonyms for word hope are ['despair']
Word hope[VBP] replaced with come[VBP]
Word hope[VBP] replaced with wish[VBP]
Word hope[VBP] replaced with despair[VBP]
Some antonyms for word increase are ['decrement', 'decrease']
Word increase[VB] replaced with increases[VB]
Word increase


  remember my bum leg strikes back this time its serious :-0.4215

Some antonyms for word strikes are ['miss']
Word strikes[VBZ] replaced with attacks[VBZ]
Word strikes[VBZ] replaced with airstrikes[VBZ]
Word strikes[VBZ] replaced with raids[VBZ]
Some antonyms for word serious are ['playful', 'frivolous']
Word serious[JJ] replaced with possible[JJ]
Word serious[JJ] replaced with severe[JJ]
Word serious[JJ] replaced with frivolous[JJ]



  cool i will. their are all kinds of complaints about this laptop online about overheating but no recalls :-0.4588

Some antonyms for word cool are ['warm', 'heat']
Word cool[NN] replaced with warm[NN]
Word cool[NN] replaced with cold[NN]
Word cool[NN] replaced with chill[NN]
Word cool[NN] replaced with warm[NN]
Word cool[NN] replaced with heat[NN]
No antonyms found for word complaints
Word complaints[NNS] replaced with inquiries[NNS]
Word complaints[NNS] replaced with allegations[NNS]
Word complaints[NNS] replaced with accusations[NNS]
Word complaints[NNS] replaced with cases[NNS]
Some antonyms for word no are ['all', 'yes']
Word no[DT] replaced with any[DT]
Word no[DT] replaced with all[DT]



 emily will be glad when mommy is done training at her new job. she misses her.  :0.2732

Some antonyms for word glad are ['sad']
Word glad[JJ] replaced with thankful[JJ]
Word glad[JJ] replaced with happy[JJ]
Word glad[JJ] replaced with sad[JJ]
Some antonyms for word misses are ['have', 'attend', 'hit']
Word misses[VBZ] replaced with throws[VBZ]
Word misses[VBZ] replaced with chances[VBZ]
Word misses[VBZ] replaced with kicks[VBZ]



 would rather the first party send bad messages than the 3rd party send mixed ones  sophmore year all over again:0.2263

No antonyms found for word party
Word party[NN] replaced with opposition[NN]
Word party[NN] replaced with coalition[NN]
Word party[NN] replaced with leader[NN]
Some antonyms for word bad are ['good', 'unregretful', 'goodness']
Word bad[JJ] replaced with good[JJ]
Word bad[JJ] replaced with good[JJ]
Word bad[JJ] replaced with unregretful[JJ]
No antonyms found for word party
Word party[NN] replaced with opposition[NN]
Word party[NN] replaced with coalition[NN]
Word party[NN] replaced with leader[NN]


...


  shame to hear this stephan :-0.4767

Some antonyms for word shame are ['honor']
Word shame[NN] replaced with disgrace[NN]
Word shame[NN] replaced with pity[NN]
Word shame[NN] replaced with embarrassment[NN]
Word shame[NN] replaced with sorrow[NN]
Word shame[NN] replaced with sadness[NN]
Word shame[NN] replaced with honor[NN]



 was intending to finish editing my 536page novel manuscript tonight but that will probably not happen. and only 12 pages are left :0.1655

No antonyms found for word novel
No POS words found for word novel with tag JJ
 -- No POS approved words! -- for word novel
 some non-POS:['novels', 'fiction', 'book', 'adaptation']
 -- No new Strings generated ---



 laid around too much today... now my head hurts :-0.4767

Some antonyms for word hurts are ['be_well']
Word hurts[NNS] replaced with undermines[NNS]
Word hurts[NNS] replaced with bothers[NNS]




No tweets found when trying to do Positive=False



  i still have not read the 9thamp10th princess diaries  saving francesca made me cry at the end. hmm those are easy books.:-0.0516

Some antonyms for word cry are ['laugh']
Word cry[NN] replaced with shout[NN]
Word cry[NN] replaced with scream[NN]
Word cry[NN] replaced with laugh[NN]
Word cry[NN] replaced with laugh[NN]
Some antonyms for word easy are ['uneasy', 'quickly', 'difficult']
Word easy[JJ] replaced with quick[JJ]
Word easy[JJ] replaced with enough[JJ]
Word easy[JJ] replaced with find[JJ]
Word easy[JJ] replaced with uneasy[JJ]
Word easy[JJ] replaced with difficult[JJ]



 my mom might have breast cancer will not find out anything for. like a week i am so worried! :-0.7234

No antonyms found for word cancer
Word cancer[NN] replaced with breast[NN]
Word cancer[NN] replaced with prostate[NN]
Word cancer[NN] replaced with disease[NN]
Some antonyms for word like are ['dislike', 'unlike', 'unalike']
Word like[IN] replaced with unlike[IN]
Word like[IN] replaced with unalike[IN]
Some antonyms for word worried are ['reassure']
Word worried[JJ] replaced with worry[JJ]
Word worried[JJ] replaced with concerned[JJ]
Word worried[JJ] replaced with wary[JJ]
Word worried[JJ] replaced with anxious[JJ]
Word worried[JJ] replaced with afraid[JJ]
Word worried[JJ] replaced with reassure[JJ]



 going to sleep. hoping tomorrow is a better day. :0.6908

Some antonyms for word hoping are ['despair']
No POS words found for word hoping with tag VBG
 -- No POS approved words! -- for word hoping
 some non-POS:['hopes', 'hoped', 'hope', 'chance']
Some antonyms for word better are ['bad', 'worsen', 'evil']
No POS words found for word better with tag JJR
 -- No POS approved words! -- for word better
 some non-POS:['good', 'need', 'get', 'make']
 -- No new Strings generated ---



  lol.. wish they understood daylight savings has ended though and breakfast is an hour later  they keep waking the kids up too:0.4019

Some antonyms for word wish are ['begrudge']
Word wish[NN] replaced with tell[NN]


No tweets found when trying to do Positive=True




## Testing Code


In [8]:
specificWord = "good"
def testOneWord(word=""):
    if(word==""):
        return
    print(word)

## Main Cell


In [12]:
specificString = ""
def specificString(textString=""):
    if(textString == "" or textString == None):
        return
    mainSentiment = senty.polarity_scores(textString)['compound']
    if(mainSentiment == 0):
        print("{} \n No sentiment found in sentence".format(textString));
        return;
    print("\n {0}:{1}\n".format(textString,mainSentiment))   
    sentenceObj = Sentence(textString, mainSentiment)
    sentenceObj = getAlternativeSentences(sentenceObj)
    alternateTweets = (sentenceObj.alternateSentences)[:]
    if(alternateTweets == [] or alternateTweets == None):
        print(" -- No new Strings generated ---\n\n")
        return
    sentenceObj = shiftSentiment(sentenceObj, True);
    printStrings(sentenceObj)
    sentenceObj = shiftSentiment(sentenceObj, False);
    printStrings(sentenceObj)
    
# specificString("Bristol University has a wonderful worklife balance")
# specificString("Bristol University has an incredibly good worklife balance")
# specificString("Bristol University has a good worklife balance")
# specificString("Bristol University has an incredible worklife balance")
# specificString("This was hard but fun to make!")

global VERBOSE_PRINTING
VERBOSE_PRINTING = True
# specificString("I hate bilal i think he's an idiot and I dislike his company")
specificString("I really hate hot chocolate, but i enjoy hot coffee!")



 I really hate hot chocolate, but i enjoy hot coffee!:0.4759

Some antonyms for word hate are ['love']
Word hate[JJ] replaced with racist[JJ]
Word hate[JJ] replaced with fear[JJ]
Some antonyms for word enjoy are ['suffer']
Word enjoy[VBP] replaced with prefer[VBP]
Word enjoy[VBP] replaced with feel[VBP]
Word enjoy[VBP] replaced with want[VBP]
Word enjoy[VBP] replaced with suffer[VBP]



 I really fucking hate this, what a piece of shit:-0.8347

Some antonyms for word hate are ['love']
Word hate[NN] replaced with racist[NN]
Word hate[NN] replaced with racism[NN]
Word hate[NN] replaced with anyone[NN]
Word hate[NN] replaced with fear[NN]
No antonyms found for word shit
Word shit[NN] replaced with fuck[NN]
Word shit[NN] replaced with crap[NN]
Word shit[NN] replaced with kinda[NN]
Word shit[NN] replaced with damn[NN]
